# Link Tracker Analytics

This notebook provides detailed analytics and visualizations for the URL shortener application.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set style for better-looking plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data from Database

In [ ]:
# Connect to database
conn = sqlite3.connect('link_tracker.db')

# Load links data
links_df = pd.read_sql_query("""
    SELECT l.*, COUNT(c.id) as click_count
    FROM links l
    LEFT JOIN clicks c ON l.id = c.link_id
    GROUP BY l.id
""", conn)

# Load clicks data
clicks_df = pd.read_sql_query("""
    SELECT c.*, l.short_code, l.original_url
    FROM clicks c
    JOIN links l ON c.link_id = l.id
""", conn)

# Convert timestamp columns to datetime
if not links_df.empty:
    links_df['created_at'] = pd.to_datetime(links_df['created_at'])

if not clicks_df.empty:
    clicks_df['clicked_at'] = pd.to_datetime(clicks_df['clicked_at'])

print(f"Total Links: {len(links_df)}")
print(f"Total Clicks: {len(clicks_df)}")

## Overview Statistics

In [ ]:
if not links_df.empty:
    print("=" * 50)
    print("LINK STATISTICS")
    print("=" * 50)
    print(f"Total number of links created: {len(links_df)}")
    print(f"Total number of clicks: {links_df['click_count'].sum()}")
    print(f"Average clicks per link: {links_df['click_count'].mean():.2f}")
    print(f"Most popular link: {links_df.loc[links_df['click_count'].idxmax(), 'short_code']} ({links_df['click_count'].max()} clicks)")
    print(f"Links with 0 clicks: {len(links_df[links_df['click_count'] == 0])}")
else:
    print("No links in database yet.")

## Top 10 Most Clicked Links

In [ ]:
if not links_df.empty:
    top_links = links_df.nlargest(10, 'click_count')[['short_code', 'original_url', 'click_count']]
    display(top_links)
    
    # Bar chart of top 10 links
    plt.figure(figsize=(12, 6))
    plt.barh(top_links['short_code'], top_links['click_count'], color='#667eea')
    plt.xlabel('Number of Clicks')
    plt.ylabel('Short Code')
    plt.title('Top 10 Most Clicked Links')
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization.")

## Click Distribution Over Time

In [ ]:
if not clicks_df.empty:
    # Group clicks by date
    clicks_by_date = clicks_df.groupby(clicks_df['clicked_at'].dt.date).size()
    
    plt.figure(figsize=(14, 6))
    plt.plot(clicks_by_date.index, clicks_by_date.values, marker='o', linewidth=2, markersize=6, color='#667eea')
    plt.xlabel('Date')
    plt.ylabel('Number of Clicks')
    plt.title('Clicks Over Time')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No click data available.")

## Hourly Click Distribution

In [ ]:
if not clicks_df.empty:
    # Extract hour from timestamp
    clicks_df['hour'] = clicks_df['clicked_at'].dt.hour
    clicks_by_hour = clicks_df.groupby('hour').size()
    
    plt.figure(figsize=(14, 6))
    plt.bar(clicks_by_hour.index, clicks_by_hour.values, color='#764ba2', alpha=0.7)
    plt.xlabel('Hour of Day (24-hour format)')
    plt.ylabel('Number of Clicks')
    plt.title('Click Distribution by Hour of Day')
    plt.xticks(range(0, 24))
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
else:
    print("No click data available.")

## Top Referrers

In [ ]:
if not clicks_df.empty:
    # Get top referrers (excluding None/empty)
    referrers = clicks_df[clicks_df['referrer'].notna() & (clicks_df['referrer'] != '')]
    
    if not referrers.empty:
        top_referrers = referrers['referrer'].value_counts().head(10)
        
        print("Top 10 Referrers:")
        display(pd.DataFrame({'Referrer': top_referrers.index, 'Count': top_referrers.values}))
        
        plt.figure(figsize=(12, 6))
        plt.barh(range(len(top_referrers)), top_referrers.values, color='#667eea')
        plt.yticks(range(len(top_referrers)), [r[:50] + '...' if len(r) > 50 else r for r in top_referrers.index])
        plt.xlabel('Number of Clicks')
        plt.ylabel('Referrer')
        plt.title('Top 10 Referrers')
        plt.tight_layout()
        plt.show()
    else:
        print("No referrer data available.")
        
    # Count direct visits (no referrer)
    direct_visits = len(clicks_df[clicks_df['referrer'].isna() | (clicks_df['referrer'] == '')])
    print(f"\nDirect visits (no referrer): {direct_visits} ({direct_visits/len(clicks_df)*100:.1f}%)")
else:
    print("No click data available.")

## Browser/User Agent Analysis

In [ ]:
if not clicks_df.empty:
    # Simple browser detection
    def detect_browser(user_agent):
        if not user_agent:
            return 'Unknown'
        ua_lower = user_agent.lower()
        if 'chrome' in ua_lower and 'edg' not in ua_lower:
            return 'Chrome'
        elif 'firefox' in ua_lower:
            return 'Firefox'
        elif 'safari' in ua_lower and 'chrome' not in ua_lower:
            return 'Safari'
        elif 'edg' in ua_lower:
            return 'Edge'
        elif 'opera' in ua_lower or 'opr' in ua_lower:
            return 'Opera'
        else:
            return 'Other'
    
    clicks_df['browser'] = clicks_df['user_agent'].apply(detect_browser)
    browser_counts = clicks_df['browser'].value_counts()
    
    print("Browser Distribution:")
    display(pd.DataFrame({'Browser': browser_counts.index, 'Count': browser_counts.values}))
    
    # Pie chart
    plt.figure(figsize=(10, 8))
    colors = ['#667eea', '#764ba2', '#f093fb', '#4facfe', '#43e97b', '#fa709a']
    plt.pie(browser_counts.values, labels=browser_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title('Browser Distribution')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
else:
    print("No click data available.")

## Unique Visitors Analysis

In [ ]:
if not clicks_df.empty:
    unique_ips = clicks_df['ip_address'].nunique()
    total_clicks = len(clicks_df)
    
    print(f"Total Clicks: {total_clicks}")
    print(f"Unique IP Addresses: {unique_ips}")
    print(f"Average clicks per visitor: {total_clicks/unique_ips:.2f}")
    
    # Top IP addresses
    top_ips = clicks_df['ip_address'].value_counts().head(10)
    print("\nTop 10 Most Active IP Addresses:")
    display(pd.DataFrame({'IP Address': top_ips.index, 'Clicks': top_ips.values}))
else:
    print("No click data available.")

## Recent Activity

In [ ]:
if not clicks_df.empty:
    recent_clicks = clicks_df.nlargest(20, 'clicked_at')[['clicked_at', 'short_code', 'original_url', 'ip_address']]
    print("Last 20 Clicks:")
    display(recent_clicks)
else:
    print("No click data available.")

## Export Summary Report

In [ ]:
if not links_df.empty:
    # Create a summary report
    summary = {
        'Total Links': len(links_df),
        'Total Clicks': links_df['click_count'].sum(),
        'Average Clicks per Link': links_df['click_count'].mean(),
        'Unique Visitors': clicks_df['ip_address'].nunique() if not clicks_df.empty else 0,
        'Generated At': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    summary_df = pd.DataFrame([summary])
    
    # Save to CSV
    summary_df.to_csv('analytics_summary.csv', index=False)
    links_df.to_csv('links_export.csv', index=False)
    if not clicks_df.empty:
        clicks_df.to_csv('clicks_export.csv', index=False)
    
    print("Reports exported successfully!")
    print("- analytics_summary.csv")
    print("- links_export.csv")
    print("- clicks_export.csv")
else:
    print("No data to export.")

In [ ]:
# Close database connection
conn.close()